In [1]:
# Google Colab notebook that can be used to run trials reported on in "RECURSIVE CONSTRUCTION OF STABLE ASSEMBLIES OF RECURRENT NEURAL NETWORKS"
# major user settings are found in the next two cells after this
# desired trial can then be run by simply running all cells
# code expects GPU runtime, efficiency may vary depending on assigned GPU

# existing output seen in later cells is from trial #10 of the CIFAR10 repeatability experiments

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.jit as jit
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, Subset
from torch._utils import _accumulate
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split 
from tqdm.auto import tqdm

import numpy as np
import numpy.random as npr
import scipy.sparse
import scipy.integrate

import pandas as pd
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#!python -V This is to check the python version

!python -m pip uninstall torch --yes
!pip install torch==1.11.0

Found existing installation: torch 1.11.0
Uninstalling torch-1.11.0:
  Successfully uninstalled torch-1.11.0
  Using cached torch-1.11.0-cp39-cp39-manylinux1_x86_64.whl (750.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0+cu116 requires torch==1.12.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.12.0+cu116 requires torch==1.12.0, but you have torch 1.11.0 which is incompatible.


In [3]:
# major model settings - by default best performing settings
task = "cifar10" # will accept cifar10, pmnist, or mnist
ns = [32 for x in range(16)] # list of integers, with each integer being the size for a component RNN. the number of component RNNs is then the length of the list
rnn_density = 0.033 # density setting for each component RNN
pre_scalar = 30.0 # sample non-zero entries for potential component RNN uniformly from -pre_scalar to pre_scalar, keep only if satisfies theorem 1
# (note that if density and pre_scalar settings are both too large code will stall because it will be impossible to find component RNNs meeting the condition)
post_scalar = 0.2 # multiply chosen component RNNs by post_scalar before beginning training (should be a positive number <= 1)

# training settings - uses Adam optimizer
num_epochs = 200 #updated by Laura to 1 from original of 200 by the research team
lr = 1e-3
lr_scale_epochs = [140, 190] # the epochs after which to perform a learning rate cut
lr_scalar = 0.1 # the scalar to multiply the learning rate by at the specified epochs
weight_decay = 1e-5

# adjacency matrix settings - this is experimental
A_sparsity = 0 # this will be fraction of possible negative feedback connections between component RNNs that are set to 0

#### Personal Note 12/07/22, added by Laura De Los Santos
I updated the number of epochs, this will probably be an issue with the replicability point, but I do not have the capacity to leave it running for that long. 

In [4]:
# settings for saving model stats and checkpoints using a specified Google Drive account
# model checkpoint is saved after each epoch, as well as an updated CSV with per-epoch training loss and test accuracy

#from google.colab import drive
#drive.mount('/content/drive')

# output file path info
output_root = "/notebooks/Modelshare" # folder path everything will be saved to
model_name = "Test" # specific trial name to use as part of each file's save name

In [5]:
# use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print('Default tensor type is now cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

Default tensor type is now cuda


In [6]:
# functions for initializing the sparse matrices

# function that checks the Theorem 1 condition for an input square matrix
def check_cond(W):
    W_diag_only = np.diag(np.diag(W))
    W_diag_pos_only = W_diag_only.copy()
    W_diag_pos_only[W_diag_pos_only < 0] = 0.0
    W_abs_cond = np.abs(W - W_diag_only) + W_diag_pos_only
    max_eig_abs_cond = np.max(np.real(np.linalg.eigvals(W_abs_cond)))
    if max_eig_abs_cond < 1:
        return True
    else:
        return False

# sampling function to use for each non-zero element in a generated matrix
# (uniform but between -1 and 1 instead of default 0 to 1)
def uniform_with_neg(x):
    return np.random.uniform(low=-1.0,high=1.0, size=x)

# function that creates a square matrix of a given size with a given density and distribution (+ scalar for the distribution)
# also zeroes out the diagonal after generation
# the uniform_with_neg function is used as the sampling function by default
def generate_random_sparse_matrix(num_units, density, dist_multiplier, dist_func=uniform_with_neg):
    test = scipy.sparse.random(num_units,num_units,density=density,format='csr',data_rvs=dist_func)
    np_test = test.toarray()
    np_test = dist_multiplier * np_test
    np.fill_diagonal(np_test, 0)
    return np_test

# create a full set of RNN modules using the above functions
# will only keep a given random RNN if it meets the condition for theorem 1
def create_modules(module_sizes, density, dist_multiplier, post_select_multiplier):
    modules = []
    for m in module_sizes:
        okay = False
        while not okay:
            cur_matrix = generate_random_sparse_matrix(m, density, dist_multiplier)
            okay = check_cond(cur_matrix)
        cur_matrix = post_select_multiplier * cur_matrix # once reach here the current matrix is one of the ones selected
        modules.append(cur_matrix)
    return modules

# function to put together the generated component RNNs into one big block diagonal weight matrix
def combine_W(W_list):
    shapes = [w.shape[0] for w in W_list]
    total_size = np.sum(shapes)
    full_W = np.zeros((total_size, total_size))
    for i in range(len(W_list)):
        cur_W = W_list[i]
        cur_size = shapes[i]
        first_index = int(np.sum(shapes[0:i]))
        last_index = first_index + cur_size
        full_W[first_index:last_index, first_index:last_index] = cur_W
    return full_W

# use theorem 1 + info about linear stability to find a metric for a given weight matrix (expected to be generated from the above)
def find_M(W_inp):
    # what we actually want to find metric for is W - I -> just W won't be stable here
    # also first need to focus on abs(W), not W itself! that is what linear stable test can find, the same metric will then work for the other (per Thm 1)
    W = np.abs(W_inp) # diagonal is set to 0 already so no need to worry about that
    W = W - np.identity(W.shape[0])
    # this just finds some M that will work, could be many others
    Q = np.identity(W.shape[0])
    # solve for M in -Q = M * W + np.transpose(W) * M
    # using integration formula for LTI system
    P = np.zeros(W.shape)
    for i in range(W.shape[0]):
        # integrating elementwise
        # keep off-diags as 0 to save time with larger martrix, as know there will be some diagonal metric, expect good odds that will find one with Q = I
        # will confirm the metric works before moving forward though (done in final function below), to be sure with stability guarantee
        def func_to_integrate(t):
            og_func = np.exp(np.transpose(W) * t) * Q * np.exp(W * t)
            return og_func[i, i]
        P[i,i] = scipy.integrate.quad(func_to_integrate, 0, np.inf)[0]
    if np.max(np.linalg.eigvals(P)) <= 0:
        # guaranteed M will be symmetric as it is definitely diagonal here, but also need it be PD for it to be a valid metric
        # should never reach this in theory, but add as a check to be safe
        print("returned metric not PD, problem!")
        return None
    return P

# replaced version of this function with more efficient one when I started using this notebook for the final version of the CIFAR10 experiment repetitions
# put everything together to get W and M to perform the training with
# W is of course part of the network
# M is used in finding negative feedback connections between components of W that maintain stability
# neither are themselves updated over the course of training
def generate_initial_W_M(module_sizes, density, dist_multiplier, post_select_multiplier):
    individual_networks = create_modules(module_sizes, density, dist_multiplier, post_select_multiplier)
    full_W = combine_W(individual_networks)

    # in prior experiments 0.5 * I was metric generally found
    # so just use that if it will work - only integrate if necessary
    matching_M = 0.5*np.identity(full_W.shape[0])
    check_formula = matching_M * (np.abs(full_W) - np.identity(full_W.shape[0])) + np.transpose(np.abs(full_W) - np.identity(full_W.shape[0])) * matching_M
    if np.max(np.linalg.eigvals(check_formula)) >= 0:
        matching_M = find_M(full_W)

        # confirm that M does work to satisfy the Theorem 1 condition with this W
        check_formula = matching_M * (np.abs(full_W) - np.identity(full_W.shape[0])) + np.transpose(np.abs(full_W) - np.identity(full_W.shape[0])) * matching_M
        if np.max(np.linalg.eigvals(check_formula)) >= 0:
            print("problem with found metric!")
            return None

    # return the final W and M
    # need these to be tensors for computation to work - but they aren't parameters!
    return torch.from_numpy(full_W).float().cuda(), torch.from_numpy(matching_M).float().cuda()

In [7]:
# helper functions for training

def add_channels(X):
    # reshaping necessary when loading the training data
    if len(X.shape) == 2:
        return X.reshape(X.shape[0], 1, X.shape[1], 1)
    elif len(X.shape) == 3:
        return X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])
    else:
        return "dimenional error"
 
def exp_lr_scheduler(epoch,
                     optimizer,
                     strategy='normal',
                     decay_eff=0.1,
                     decayEpoch=[]):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""
 
    if strategy == 'normal':
        if epoch in decayEpoch:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_eff
            print('New learning rate is: ', param_group['lr'])
    else:
        print('wrong strategy')
        raise ValueError('A very specific bad thing happened.')
 
    return optimizer
 
# get adjacency matrix specifying which modules should be in negative feedback with each other
# for full negative feedback frac_zeros would be 0, but will play with sparsity settings
# (this is part of initialization)
def create_random_A(ns,frac_zeros):
    num_networks = len(ns)
    A = torch.cuda.FloatTensor(num_networks, num_networks).uniform_() > frac_zeros
    A_tril = torch.tril(A)
    A_tril.fill_diagonal_(0)
    # note only lower triangular needs to be trained, as bidirectional version of connection determined by negative feedback stability cond
    return A_tril
 
def create_mask_given_A(A,ns):
    '''
    Creates 'hidden' mask for training, given an arbitrary adjacency matrix.
    
    ARGS:
        - A: adjacency matrix
        - ns: list of neural population sizes (e.g ns = [5,4,18,2]).
    OUTS:
        - A: mask
    '''

    N_nets = len(ns)
    mask = []
    
    for i in range(N_nets):
            mask_row_i = torch.cat([torch.ones((ns[i],ns[j])) if A[i,j] == 1 and i >= j else torch.zeros((ns[i],ns[j]))  for j in range(N_nets)],1)
            mask.append(mask_row_i)    
    
    final = torch.cat(mask,0)
    final.to(device)
    return final

In [8]:
# get one of the 3 sequential image classification datasets we use
# (sequential MNIST, permuted sequential MNIST, and sequential CIFAR10 - accepted arguments are mnist, pmnist, and cifar10 respectively)
# always uses the same batch size settings, 64 for train and 1024 for test
def getData(name, train_bs=64, test_bs=1024):

    if name == 'cifar10':
        transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

        transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

        train_loader = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

        offset = 2000
        rng = np.random.RandomState(1234)
        R = rng.permutation(len(train_loader))
        lengths = (len(train_loader) - offset, offset)
        train_loader, val_loader = [Subset(train_loader, R[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]

        # specifying the generator here is necessary for the code to work on colab
        generator = torch.Generator(device=device)
        generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
        train_loader = torch.utils.data.DataLoader(train_loader, batch_size=train_bs, shuffle=True, generator=generator)
        val_loader = torch.utils.data.DataLoader(val_loader, batch_size=test_bs, shuffle=False, generator=generator)
        testset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform_test)
        test_loader = torch.utils.data.DataLoader(testset, batch_size=test_bs, shuffle=False, generator=generator)

    if name == 'mnist':
        train_loader = datasets.MNIST('./data', train=True, download=True,
                           transform=transforms.Compose([transforms.ToTensor(),]))
 
        val_loader = datasets.MNIST('./data', train=True, download=True,
                           transform=transforms.Compose([transforms.ToTensor(),]))
 
        offset = 2000
        rng = np.random.RandomState(1234)
        R = rng.permutation(len(train_loader))
        lengths = (len(train_loader) - offset, offset)
        train_loader, val_loader = [Subset(train_loader, R[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]
 
        # specifying the generator here is necessary for the code to work on colab
        generator = torch.Generator(device=device)
        generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
        train_loader = torch.utils.data.DataLoader(train_loader, batch_size=train_bs, shuffle=True, generator=generator)
        val_loader = torch.utils.data.DataLoader(val_loader, batch_size=test_bs, shuffle=False, generator=generator)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./data', train=False, download=False,
            transform=transforms.Compose([transforms.ToTensor(),])),
            batch_size=test_bs, shuffle=False, generator=generator)
 
    if name == 'pmnist':
        trainset = datasets.MNIST(root='./data', train=True, download=True,
                            transform=transforms.Compose([transforms.ToTensor(),]))
        testset = datasets.MNIST(root='./data', train=False, download=False,
                            transform=transforms.Compose([ transforms.ToTensor(),]))
        
        x_train = trainset.train_data
        y_train = trainset.targets
        x_test = testset.test_data        
        y_test = testset.targets
 
        torch.manual_seed(12008)        
        perm = torch.randperm(784)

        x_train_permuted = x_train.reshape(x_train.shape[0],-1)
        x_train_permuted = x_train_permuted[:, perm]
        x_train_permuted = x_train_permuted.reshape(x_train.shape[0], 28, 28)
        
        x_test_permuted = x_test.reshape(x_test.shape[0],-1)
        x_test_permuted = x_test_permuted[:, perm]
        x_test_permuted = x_test_permuted.reshape(x_test.shape[0], 28, 28)        
 
        x_train_permuted = add_channels(x_train_permuted)
        x_test_permuted = add_channels(x_test_permuted)
        
        train_loader = torch.utils.data.TensorDataset(x_train_permuted.float(), y_train)
        offset = 2000
        rng = np.random.RandomState(1234)
        R = rng.permutation(len(train_loader))
        lengths = (len(train_loader) - offset, offset)
        train_loader, val_loader = [Subset(train_loader, R[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]
 
        # specifying the generator here is necessary for the code to work on colab
        generator = torch.Generator(device=device)
        generator.manual_seed(int(torch.empty((), dtype=torch.int64).random_().item()))
        train_loader = torch.utils.data.DataLoader(train_loader, batch_size=train_bs, shuffle=True, generator=generator)
        val_loader = torch.utils.data.DataLoader(val_loader, batch_size=test_bs, shuffle=False, generator=generator) 
        test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_permuted.float(), y_test),
                                                batch_size=test_bs, shuffle=False, generator=generator)
 
    return train_loader, val_loader, test_loader

In [9]:
# load specified dataset
train_loader,val_loader,test_loader = getData(task)

Files already downloaded and verified


In [10]:
# class definitions for the sparse combo networks

class rnnAssemblyCell_Thm1(jit.ScriptModule):
    '''
    Pytorch module for training the following system:
        tau*dx/dt = -x + W*phi(x) + L*x+ u(t) 
    where tau > 0, phi is a nonlinearity, W is block diagonal, L is some 'contracting' combination matrix and u is some input. 
       
    ''' 
    
    def __init__(self, input_size, hidden_sizes, output_size, alpha, A, density, pre_select_mult, post_select_mult):
        super(rnnAssemblyCell_Thm1, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.hidden_size = int(np.sum(hidden_sizes))
        self.output_size = output_size
        self.alpha = alpha 
        
        # initialize linear input and output layers, to be trained, along with biases
        self.weight_ih = nn.Parameter(torch.normal(0,1/np.sqrt(self.hidden_size),(self.hidden_size, self.input_size)))        
        self.weight_ho = nn.Parameter(torch.normal(0,1/np.sqrt(self.hidden_size),(self.output_size, self.hidden_size)))
        self.bias_oh = nn.Parameter(torch.normal(0,1/np.sqrt(self.hidden_size),(1,self.output_size)))
        self.bias_hh = nn.Parameter(torch.normal(0,1/np.sqrt(self.hidden_size),(1,self.hidden_size)))               
 
        # specify W and M here based on the random initialization mentioned     
        self.W, self.M = generate_initial_W_M(self.hidden_sizes, density, pre_select_mult, post_select_mult)
        self.M_inv = torch.inverse(self.M)
 
        # L contains the connections between subsystems. this will be trained. 
        self.L_mask = create_mask_given_A(A,self.hidden_sizes).bool()
        self.L_train = nn.Parameter(self.L_mask*torch.normal(0,1/np.sqrt(np.mean(self.hidden_sizes)),(np.sum(self.hidden_sizes), np.sum(self.hidden_sizes))))
        
    @jit.script_method
    def forward(self, input, state):
        # type: (Tensor, Tensor) -> Tuple[Tensor, Tensor]
        L_masked = self.L_train*self.L_mask
        
        fx = -state + F.relu(state @ self.W.T + input @ (self.weight_ih.T) + self.bias_hh) + state @ (L_masked.T - (self.M @ L_masked) @ self.M_inv) 
              
        hx =  state + self.alpha*fx
        
        hy = hx @ (self.weight_ho.T)
 
        return hy, hx
 
class rnnAssemblyLayer_Thm1(jit.ScriptModule):
    def __init__(self, cell, *cell_args):
        super(rnnAssemblyLayer_Thm1, self).__init__()
        self.cell = cell(*cell_args)
 
    @jit.script_method
    def forward(self, input):
        # type: (Tensor) -> Tuple[Tensor, Tensor]
        state = 0*0.1*torch.randn(input.shape[0],self.cell.hidden_size, device='cuda')
        
        inputs = input.unbind(1)              
        
        outputs = torch.jit.annotate(List[Tensor], [])
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
 
        return torch.stack(outputs).permute(1,0,2), state

In [11]:
# training function for sequential CIFAR
# next cell contains the slightly different function for when training is on one of the MNIST tasks instead
def train_CIFAR(rnn,optimizer,train_loader,test_loader,max_epoch,decay_eff,decay_epochs):    
    '''
    Main training loop.
    '''
 
    optim_params = list(rnn.parameters())
    criterion = nn.CrossEntropyLoss()
    test_accs = []
    train_losses = []
    epochs_list = [] # just grabbing numbers for the sake of dataframe
 
    # Train for some epochs
    for epoch in tqdm(range(max_epoch),total = max_epoch):
        rnn.train()
        loss_epoch = []
        for i,(inp,target) in tqdm(enumerate(train_loader),total = len(train_loader)):    
            optimizer.zero_grad()           
            
            # had to add these lines for colab GPU
            inp = inp.cuda() 
            target = target.cuda()

            # CIFAR-10 is 32x32, so 1024 pixels
            # it is also color, so the original format would be (batch_size, 3, 32, 32)
            # for the RNN input, want input size to be the last variable though
            output,_ = rnn(inp.view(-1, 3, int(1024)).permute(0,2,1))
            
            loss = criterion(output[:,-1,:], target.to(device))
            loss_epoch.append(loss.item())
            loss.backward()        
            optimizer.step()
 
        # track loss over time, and have it reflect the mean loss over the batches so it is more reflective of training trends than last batch loss
        mean_loss = np.mean(loss_epoch)
        train_losses.append(mean_loss)

        # calling the network with no training epoch 0, want each epoch number to reflect how many have been run so far, so add 1 here
        print('Epoch {}, mean batch loss {}'.format(epoch+1, mean_loss))

        # use specified scalar to learning rate after specified epochs
        # (subtracting 1 from each input epoch to again account for epoch naming convention)
        optimizer = exp_lr_scheduler(epoch, optimizer, decay_eff=decay_eff, decayEpoch=[x-1 for x in decay_epochs])
 
        # testing is fast so no problem doing it every epoch 
        rnn.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            for inp,target in test_loader:
                inp = inp.cuda() # adding here too
                target = target.cuda()
                # reformat input in same way for CIFAR here
                output,_ = rnn(inp.view(-1, 3, int(1024)).permute(0,2,1))
 
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(output[:,-1,:].data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
            print('Accuracy of the network on the 10000 test images: %d %%' % (
                  100 * correct / total))
            test_accs.append((100.0 * float(correct) / float(total))) # for the actual list make sure we are getting exact test accuracy, so convert to floats!
        epochs_list.append(epoch+1) # end of epoch so label +1

        # save model for every epoch as storage space required is quite small, can have training disruptions with colab
        model_path = os.path.join(output_root, model_name + "-cifar10-" + "epoch" + str(epoch+1) + ".pt") # end of epoch so label +1
        rnn.save(model_path) # would use torch.jit.load to reload in the future

        # save stats so far, will overwrite every time as rows are added to dataframe
        stats_path = os.path.join(output_root, model_name + "-cifar10-stats.csv")
        cur_stats = pd.DataFrame()
        cur_stats["epoch"] = epochs_list
        cur_stats["loss"] = train_losses
        cur_stats["test-acc"] = test_accs
        cur_stats.to_csv(stats_path, index=False)
        
    return rnn, optimizer, test_accs, train_losses

In [12]:
# analogous training function for sequential or permuted MNIST
def train_MNIST(rnn,optimizer,train_loader,test_loader,max_epoch,decay_eff,decay_epochs,task):    
    '''
    Main training loop.
    '''
 
    optim_params = list(rnn.parameters())
    criterion = nn.CrossEntropyLoss()
    test_accs = []
    train_losses = []
    epochs_list = [] # just grabbing numbers for the sake of dataframe
 
    # Train for some epochs
    for epoch in tqdm(range(max_epoch),total = max_epoch):
        rnn.train()
        loss_epoch = []
        for i,(inp,target) in tqdm(enumerate(train_loader),total = len(train_loader)):    
            optimizer.zero_grad()           
            
            # had to add these lines for colab GPU
            inp = inp.cuda() 
            target = target.cuda()

            output,_ = rnn(inp.view(-1, int(784)).unsqueeze(dim = 2))
            
            loss = criterion(output[:,-1,:], target.to(device))
            loss_epoch.append(loss.item())
            loss.backward()        
            optimizer.step()
 
        # track loss over time, and have it reflect the mean loss over the batches so it is more reflective of training trends than last batch loss
        mean_loss = np.mean(loss_epoch)
        train_losses.append(mean_loss)

        # calling the network with no training epoch 0, want each epoch number to reflect how many have been run so far, so add 1 here
        print('Epoch {}, mean batch loss {}'.format(epoch+1, mean_loss))

        # use specified scalar to learning rate after specified epochs
        # (subtracting 1 from each input epoch to again account for epoch naming convention)
        optimizer = exp_lr_scheduler(epoch, optimizer, decay_eff=decay_eff, decayEpoch=[x-1 for x in decay_epochs])
 
        # testing is fast so no problem doing it every epoch 
        rnn.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            for inp,target in test_loader:
                inp = inp.cuda() # adding here too
                target = target.cuda()
                output,_ = rnn(inp.view(-1, int(784)).unsqueeze(dim = 2))
 
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(output[:,-1,:].data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
            print('Accuracy of the network on the 10000 test images: %d %%' % (
                  100 * correct / total))
            test_accs.append((100.0 * float(correct) / float(total))) # for the actual list make sure we are getting exact test accuracy, so convert to floats!
        epochs_list.append(epoch+1) # end of epoch so label +1

        # save model for every epoch as storage space required is quite small, can have training disruptions with colab
        model_path = os.path.join(output_root, model_name + "-" + task + "-" + "epoch" + str(epoch+1) + ".pt") # end of epoch so label +1
        rnn.save(model_path) # would use torch.jit.load to reload in the future

        # save stats so far, will overwrite every time as rows are added to dataframe
        stats_path = os.path.join(output_root, model_name + "-" + task + "-stats.csv")
        cur_stats = pd.DataFrame()
        cur_stats["epoch"] = epochs_list
        cur_stats["loss"] = train_losses
        cur_stats["test-acc"] = test_accs
        cur_stats.to_csv(stats_path, index=False)
        
    return rnn, optimizer, test_accs, train_losses

In [ ]:
'''
Define network and use main training loop
'''
 
# setup the static RNN settings now
dt_rnn = .03
tau_rnn = 1
alpha = dt_rnn/tau_rnn
output_size = 10 # always 10 possible labels

# other settings that vary only based on above settings
if task == "cifar10":
    input_size = 3 # rgb
else:
    input_size = 1 # black and white
A = create_random_A(ns, A_sparsity)

# instantiate a network and optimizer
rnn = rnnAssemblyLayer_Thm1(rnnAssemblyCell_Thm1,input_size,ns,output_size,alpha,A,rnn_density,pre_scalar,post_scalar)
rnn.to(device)
 
# setup the optimizer
optim_params = list(rnn.parameters()) 
optimizer = torch.optim.Adam(optim_params, lr=lr, weight_decay=weight_decay)
 
# save initialization of network
model_path = os.path.join(output_root, model_name + "-" + task + "-" + "epoch0.pt")
rnn.save(model_path)
 
# train the initialized network with the loaded dataset, using appropriate training function
if task == "cifar10":
    rnn,optimizer,test_accs,train_losses = train_CIFAR(rnn,optimizer,train_loader,test_loader,num_epochs,lr_scalar,lr_scale_epochs)
else:
    rnn,optimizer,test_accs,train_losses = train_MNIST(rnn,optimizer,train_loader,test_loader,num_epochs,lr_scalar,lr_scale_epochs,task)
# next two cells will then plot the found accuracies and losses

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/750 [00:00<?, ?it/s]

In [ ]:
plt.plot(test_accs)

In [ ]:
plt.plot(train_losses)